In [36]:
import re
import nltk
import string
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer() 

nltk_words = set(nltk.corpus.stopwords.words('english'))
stop_words = []
for word in nltk_words:
    stop_words.append(word.translate(str.maketrans('', '', string.punctuation)))

def preprocess(sentence):
    res = sentence.lower()
    res = res.translate(str.maketrans('', '', string.punctuation))
    tokenized_words = nltk.word_tokenize(res)
    res = [word for word in tokenized_words if word not in stop_words]
    res = [lemmatizer.lemmatize(r) for r in res]
    res = [re.sub(r"[^A-Za-z]+", '', r) for r in res]
    res = [r for r in res if len(r) > 3]
    return res

In [37]:
import pandas as pd

df = pd.read_csv('res_mul_all.csv')
df.head()

,id,reviewID,sentenceID,review,category,polarity,entity,preprocessed_sentence,type_sentence
0,0,RL#3,RL#3:1,I am not necessarily fanatical about this plac...,VALUE#PRICES,positive,VALUE,i am not necessarily fanatical about this plac...,compound_sentence
1,2,TR#2,TR#2:2,The high prices you're going to pay is for the...,VALUE#PRICES,negative,VALUE,the high prices you 're going to pay is for th...,complex_sentence
2,3,TR#2,TR#2:2,The high prices you're going to pay is for the...,VALUE#PRICES,negative,VALUE,the high prices you 're going to pay is for th...,complex_sentence
3,4,TR#2,TR#2:2,The high prices you're going to pay is for the...,VALUE#PRICES,negative,VALUE,the high prices you 're going to pay is for th...,complex_sentence
4,5,TR#2,TR#2:2,The high prices you're going to pay is for the...,VALUE#PRICES,negative,VALUE,the high prices you 're going to pay is for th...,complex_sentence


In [38]:
text_data = []
for sentence in df['review']:
    tokens = preprocess(sentence)
    text_data.append(tokens)

In [39]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]

import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [60]:
import gensim

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 20, id2word=dictionary, random_state = 42)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics()

In [61]:
all_topics = []
for topic in topics:
    for pair in topic[1].split(' + '):
        word = pair.split('*')[1].replace("\"", '')
        all_topics.append(word)
print(sorted(list(dict.fromkeys(all_topics))))

['absolutely', 'always', 'amazing', 'appropriate', 'atmosphere', 'away', 'bagel', 'beautiful', 'best', 'bland', 'cant', 'casual', 'caviar', 'cozy', 'decor', 'delicious', 'dish', 'enjoy', 'even', 'ever', 'excellent', 'family', 'filet', 'fish', 'flavorful', 'food', 'fresh', 'friendly', 'garden', 'going', 'good', 'great', 'high', 'lamb', 'like', 'lobster', 'made', 'meal', 'moderate', 'never', 'nice', 'outside', 'pasta', 'pizza', 'place', 'plentiful', 'portion', 'price', 'reasonably', 'recommend', 'relaxed', 'restaurant', 'rice', 'roll', 'rude', 'salad', 'sandwich', 'selection', 'service', 'spicy', 'staff', 'sushi', 'tempura', 'terrible', 'time', 'treated', 'value', 'view', 'waiter', 'want', 'well', 'wonderful', 'world', 'worth', 'would']


In [62]:
all_topics = []
for topic in topics:
    for pair in topic[1].split(' + '):
        word = pair.split('*')[1].replace("\"", '')
        all_topics.append(word)
print(sorted(list(dict.fromkeys(all_topics))))

['absolutely', 'always', 'amazing', 'appropriate', 'atmosphere', 'away', 'bagel', 'beautiful', 'best', 'bland', 'cant', 'casual', 'caviar', 'cozy', 'decor', 'delicious', 'dish', 'enjoy', 'even', 'ever', 'excellent', 'family', 'filet', 'fish', 'flavorful', 'food', 'fresh', 'friendly', 'garden', 'going', 'good', 'great', 'high', 'lamb', 'like', 'lobster', 'made', 'meal', 'moderate', 'never', 'nice', 'outside', 'pasta', 'pizza', 'place', 'plentiful', 'portion', 'price', 'reasonably', 'recommend', 'relaxed', 'restaurant', 'rice', 'roll', 'rude', 'salad', 'sandwich', 'selection', 'service', 'spicy', 'staff', 'sushi', 'tempura', 'terrible', 'time', 'treated', 'value', 'view', 'waiter', 'want', 'well', 'wonderful', 'world', 'worth', 'would']


In [63]:
import pandas as pd

def aspect_topic(all_topics):
    sf = pd.DataFrame(columns=['id','review','category','term'])
    count = 0
    index = 0
    res = []
    for sentence in df['review']:
        lowercased = sentence.lower()
        term = []
        category = []
        for cat in df['category'][index].split(','):
            category.append(cat.split('#')[0])
        id_name = df['id'][index]
        for topic in all_topics:
            if topic in lowercased:
                term.append(topic)
#         print(term)
        if len(term) == 0:
            print(sentence)
            count += 1
        sf = sf.append({'id': id_name, 'review': sentence.strip().lower().replace('  ', ' '), 'category': '|'.join(category), 'term': '|'.join(term)}, ignore_index=True)
        index += 1
    print(count)
    sf.to_csv("lda.csv")
    sf.to_excel("lda.xlsx")

In [59]:
aspect_topic(list(dict.fromkeys(all_topics)))

The hostess is rude to the point of being offensive.
First went here to enjoy their garden terrace.
Took my mom for Mother's Day, and the maitre d' was pretty rude.
The tuna and wasabe potatoes are bad.
Their tuna tartar appetizer is to die for.
The dining room is quietly elegant with no music to shout over -- how refreshing!
The menu is very limited - i think we counted 4 or 5 entrees.
Delivery is fast too.
Thius is a must for anyone who loves Shabu-Shabu.
Taxan horrible!
I had the worst ravioli ever.
Whether it's the parmesean porcini souffle or the lamb glazed with balsamic vinegar, you will surely be transported to Northern Italy with one bite.
I had their eggs benedict for brunch, which were the worst in my entire life, I tried removing the hollondaise sauce completely that was how failed it was.
And the Tom Kha soup was pathetic.
We had the lobster sandwich and it was fantastic.
Mizu is home to creative and unique rolls not to found anywhere else.
I ordered the smoked salmon and 

In [64]:
aspect_topic(list(dict.fromkeys(all_topics)))

I had the duck breast special on my last visit and it was not incredible.
Once you step into Cosette, you're miraculously in a small, off-the-beaten path Parisian bistro.
My wife had the fried shrimp which are huge and loved it.
There was a small wait, but shorter than I expected.
The tuna and wasabe potatoes are bad.
Ingredients are organic which is a real plus for me.
Their tuna tartar appetizer is to die for.
The menu is very limited - i think we counted 4 or 5 entrees.
We ordered the special, grilled branzino, that was so infused with bone, it was difficult to eat.
Delivery is fast too.
Thius is a must for anyone who loves Shabu-Shabu.
Taxan horrible!
I had their eggs benedict for brunch, which were the worst in my entire life, I tried removing the hollondaise sauce completely that was how failed it was.
The seats are uncomfortable if you are sitting against the wall on wooden benches.
(The asparagus, truffle oil, parmesan bruschetta is a winner!)
And the Tom Kha soup was pathetic.